<a href="https://colab.research.google.com/github/SandeeeeeeeeepDey/data-science-11-weeks-progg/blob/main/Chapter_11_pushups.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Chapter 11 handson

In [2]:
%pip install wandb
!wandb login

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.1/267.1 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.3 MB/s eta 0:00:00
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [3]:
import tensorflow as tf
import pandas as pd
import numpy as np
import wandb
from wandb.keras import WandbCallback
import math

In [4]:
(X_train_full, y_train_full), (X_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()

4422102/4422102 [==============================] - 0s 0us/step


In [5]:
zwanzig = int(len(X_train_full)*0.2)

In [10]:
X_train, y_train = X_train_full[zwanzig:], y_train_full[zwanzig:]
X_valid, y_valid = X_train_full[:zwanzig], y_train_full[:zwanzig]

In [6]:
class Modol(tf.keras.Model):
    def __init__(self, rate = 0.1, num_layers = 3, num_units = 100, activation = 'relu', kernel_initializer = "he_normal", **kwargs):
        super().__init__(**kwargs)
        self.flatten = tf.keras.layers.Flatten()
        self.dense = [tf.keras.layers.Dense(num_units, activation = activation, kernel_initializer = kernel_initializer) for layer in range(num_layers)]
        self.op = tf.keras.layers.Dense(10, activation = 'softmax')

    def call(self, input):
        input = self.flatten(input)
        for layer in self.dense:
            input = layer(input)
        return self.op(input)

def build_model(seed = 37):
    tf.random.set_seed(seed)
    return Modol()

def build_train_model(optimizer):
    early_stop = tf.keras.callbacks.EarlyStopping(patience = 5, monitor = "val_loss")
    model_checkpoint = tf.keras.callbacks.ModelCheckpoint("model", save_best_only = True)
    model = build_model()
    model.compile(loss = "sparse_categorical_crossentropy", optimizer = optimizer, metrics = ["accuracy"])
    return model.fit(X_train, y_train, epochs = 10, validation_data = (X_valid, y_valid), callbacks = [early_stop, model_checkpoint])


##power scheduling

here, the learning will go from a ligh number to a low number slowly or quickly depending on the power



```
learning_rate = initial_learning_rate / (1 + step / decay_steps)**power
```



In [ ]:
lr_scheduler = tf.keras.optimizers.schedules.InverseTimeDecay(
    initial_learning_rate = 1e-5,
    decay_steps = 10_000,
    decay_rate = 1.0, # power
    staircase = False #Discrete decay or not
)
optimizer = tf.keras.optimizers.SGD(learning_rate = lr_scheduler)

In [ ]:
power_schedule_try = build_train_model(optimizer)

Epoch 1/10
1500/1500 [==============================] - 6s 4ms/step - loss: 17.9117 - accuracy: 0.6048 - val_loss: 8.8503 - val_accuracy: 0.6813
Epoch 2/10
1500/1500 [==============================] - 6s 4ms/step - loss: 7.3181 - accuracy: 0.7022 - val_loss: 6.3016 - val_accuracy: 0.7134
Epoch 3/10
1500/1500 [==============================] - 5s 4ms/step - loss: 5.5304 - accuracy: 0.7235 - val_loss: 5.1268 - val_accuracy: 0.7254
Epoch 4/10
1500/1500 [==============================] - 6s 4ms/step - loss: 4.6416 - accuracy: 0.7336 - val_loss: 4.5429 - val_accuracy: 0.7318
Epoch 5/10
1500/1500 [==============================] - 5s 4ms/step - loss: 4.1180 - accuracy: 0.7391 - val_loss: 4.1608 - val_accuracy: 0.7390
Epoch 6/10
1500/1500 [==============================] - 6s 4ms/step - loss: 3.7510 - accuracy: 0.7444 - val_loss: 3.8317 - val_accuracy: 0.7439
Epoch 7/10
1500/1500 [==============================] - 5s 4ms/step - loss: 3.4642 - accuracy: 0.7505 - val_loss: 3.5868 - val_accuracy

## exponential scheduling



```
learning_rate = initial_learning_rate * decay_rate ** (step / decay_steps)
```



In [ ]:
lr_scheduler = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate = 1e-5,
    decay_steps = 20_000,
    decay_rate = 0.1,
    staircase = False
)
optimizer = tf.keras.optimizers.SGD(learning_rate = lr_scheduler)

In [ ]:
exponential_scheduler_try = build_train_model(optimizer)

Epoch 1/10
1500/1500 [==============================] - 6s 4ms/step - loss: 14.3893 - accuracy: 0.5851 - val_loss: 7.3421 - val_accuracy: 0.6750
Epoch 2/10
1500/1500 [==============================] - 6s 4ms/step - loss: 6.2740 - accuracy: 0.6892 - val_loss: 5.6539 - val_accuracy: 0.6998
Epoch 3/10
1500/1500 [==============================] - 6s 4ms/step - loss: 5.0493 - accuracy: 0.7137 - val_loss: 4.8137 - val_accuracy: 0.7212
Epoch 4/10
1500/1500 [==============================] - 6s 4ms/step - loss: 4.4109 - accuracy: 0.7246 - val_loss: 4.3315 - val_accuracy: 0.7312
Epoch 5/10
1500/1500 [==============================] - 5s 4ms/step - loss: 4.0067 - accuracy: 0.7316 - val_loss: 4.0679 - val_accuracy: 0.7335
Epoch 6/10
1500/1500 [==============================] - 6s 4ms/step - loss: 3.7352 - accuracy: 0.7369 - val_loss: 3.8195 - val_accuracy: 0.7357
Epoch 7/10
1500/1500 [==============================] - 5s 4ms/step - loss: 3.5355 - accuracy: 0.7400 - val_loss: 3.6833 - val_accuracy

Do i want to earn how to make custom Schedulers?

hell yeah!

1. to do that ill have to make a custom callback
2. but i can also make a normal function and feed it callbacks.LearningRateScheduler





In [ ]:
K = tf.keras.backend # to be able to access already set values ig

class ExponentialDecay(tf.keras.callbacks.Callback):
    def __init__(self, n_steps = 40000):
        super().__init__()
        self.n_steps =n_steps

    def on_batch_begin(self, batch, logs = None):
        lr = K.get_value(self.model.optimizer.learning_rate)
        new_learning_rate = lr * 0.1 ** (1 / self.n_steps)
        K.set_value(self.model.optimizer.learning_rate, new_learning_rate)

    def on_epoch_end(self, epoch, logs = None):
        logs = logs or {}
        logs["lr"] = K.get_value(self.model.optimizer.learning_rate)

In [ ]:
lr0 = 0.01
model = build_model()
optimizer = tf.keras.optimizers.SGD(learning_rate = lr0)
model.compile(loss = "sparse_categorical_crossentropy", optimizer = optimizer,
              metrics = ["accuracy"])

In [ ]:
batch_size = 32
n_epochs = 20
n_steps = n_epochs *  math.ceil(len(X_train) / batch_size) # tor=tal number of iterations
exp_decay = ExponentialDecay(n_steps)
history = model.fit(X_train, y_train, validation_data = (X_valid, y_valid), epochs = n_epochs, callbacks = [exp_decay])

Epoch 1/20
1500/1500 [==============================] - 10s 5ms/step - loss: 1109176320.0000 - accuracy: 0.0996 - val_loss: 26125066.0000 - val_accuracy: 0.0935 - lr: 0.0089
Epoch 2/20
1500/1500 [==============================] - 8s 5ms/step - loss: 2.3027 - accuracy: 0.1016 - val_loss: 26125066.0000 - val_accuracy: 0.0935 - lr: 0.0079
Epoch 3/20
1500/1500 [==============================] - 9s 6ms/step - loss: 2.3027 - accuracy: 0.1016 - val_loss: 26125066.0000 - val_accuracy: 0.0935 - lr: 0.0071
Epoch 4/20
1500/1500 [==============================] - 8s 5ms/step - loss: 2.3026 - accuracy: 0.1011 - val_loss: 26125066.0000 - val_accuracy: 0.0935 - lr: 0.0063
Epoch 5/20
1500/1500 [==============================] - 8s 5ms/step - loss: 2.3026 - accuracy: 0.1006 - val_loss: 26125066.0000 - val_accuracy: 0.0935 - lr: 0.0056
Epoch 6/20
1500/1500 [==============================] - 9s 6ms/step - loss: 2.3026 - accuracy: 0.1008 - val_loss: 26125066.0000 - val_accuracy: 0.0935 - lr: 0.0050
Epoch 

2.

In [ ]:
def exponential_decay(lr0, decay_steps):
    def exponential_decay_fn(epoch):
        return lr0 *0.1 **(epoch/decay_steps)
    return exponential_decay_fn

exponential_decay_fn = exponential_decay(lr0 = 0.01, decay_steps = 20)

then put it in callback

## piecewise constant scheduling

In [ ]:
lr_schedule = tf.keras.optimizers.schedules.PiecewiseConstantDecay(
    boundaries = [50000, 80000],
    values = [0.01, 0.005, 0.001]
)
optimizer = tf.keras.optimizers.SGD(learning_rate = lr_schedule)

In [ ]:
history_piecewise_try = build_train_model(optimizer)

Epoch 1/10
1500/1500 [==============================] - 8s 5ms/step - loss: 799413.5625 - accuracy: 0.1007 - val_loss: 2.3033 - val_accuracy: 0.0996
Epoch 2/10
1500/1500 [==============================] - 6s 4ms/step - loss: 2.3034 - accuracy: 0.0988 - val_loss: 2.3028 - val_accuracy: 0.0984
Epoch 3/10
1500/1500 [==============================] - 6s 4ms/step - loss: 2.3029 - accuracy: 0.1002 - val_loss: 2.3028 - val_accuracy: 0.1010
Epoch 4/10
1500/1500 [==============================] - 6s 4ms/step - loss: 2.3029 - accuracy: 0.1000 - val_loss: 2.3028 - val_accuracy: 0.1001
Epoch 5/10
1500/1500 [==============================] - 6s 4ms/step - loss: 2.3030 - accuracy: 0.0983 - val_loss: 2.3031 - val_accuracy: 0.1002
Epoch 6/10
1500/1500 [==============================] - 5s 4ms/step - loss: 2.3030 - accuracy: 0.0979 - val_loss: 2.3028 - val_accuracy: 0.0984
Epoch 7/10
1500/1500 [==============================] - 6s 4ms/step - loss: 2.3029 - accuracy: 0.0979 - val_loss: 2.3028 - val_accu

In [ ]:
def piecewise_constant(boundaries, values):
    boundaries = np.array([0] + boundaries)
    values = np.array(values)

    def piecewise_constant_fn(epoch):
        return values[(boundaries > epoch).argmax() - 1]

    return piecewise_constant_fn

piecewise_constant_fn = piecewise_constant([5, 15], [0.01, 0.005, 0.001])

In [ ]:
n_epochs = 20
lr_schedule = tf.keras.callbacks.LearningRateScheduler(piecewise_constant_fn)

model = build_model()

optimizer = tf.keras.optimizers.Nadam(learning_rate=lr0)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,
              metrics=["accuracy"])
history = model.fit(X_train, y_train, epochs=n_epochs,
                    validation_data=(X_valid, y_valid),
                    callbacks=[lr_schedule])

Epoch 1/20
1500/1500 [==============================] - 8s 4ms/step - loss: 3.8322 - accuracy: 0.7223 - val_loss: 0.5862 - val_accuracy: 0.7961 - lr: 0.0100
Epoch 2/20
1500/1500 [==============================] - 6s 4ms/step - loss: 0.6100 - accuracy: 0.7918 - val_loss: 0.6413 - val_accuracy: 0.7857 - lr: 0.0100
Epoch 3/20
1500/1500 [==============================] - 6s 4ms/step - loss: 0.6278 - accuracy: 0.7919 - val_loss: 0.6511 - val_accuracy: 0.7888 - lr: 0.0100
Epoch 4/20
1500/1500 [==============================] - 7s 5ms/step - loss: 0.8185 - accuracy: 0.7174 - val_loss: 1.5426 - val_accuracy: 0.3850 - lr: 0.0100
Epoch 5/20
1500/1500 [==============================] - 7s 4ms/step - loss: 1.5042 - accuracy: 0.3486 - val_loss: 1.3248 - val_accuracy: 0.3908 - lr: 0.0100
Epoch 6/20
1500/1500 [==============================] - 6s 4ms/step - loss: 1.3439 - accuracy: 0.4002 - val_loss: 1.3040 - val_accuracy: 0.4320 - lr: 0.0050
Epoch 7/20
1500/1500 [==============================] - 6s

In [16]:
K = tf.keras.backend

class PiecewiseConstDecay(tf.keras.callbacks.Callback):
    def __init__(self, boundaries, values):
        super().__init__()
        self.boundaries = boundaries
        self.values = values

    def on_epoch_begin(self, epoch, logs = None):
        # lr = K.get_value(self.optimizer.learning_rate)
        new_lr = 0
        for i, boundary in enumerate(self.boundaries):
            if epoch <= boundary:
                new_lr = self.values[i]
            else:
                if i == len(self.boundaries) - 1:
                    new_lr = self.values[i+1]
                else:
                    continue
        if new_lr != 0:
            K.set_value(self.model.optimizer.learning_rate, new_lr)

    def on_epoch_end(self, epoch, logs = None):
        logs = logs or {}
        logs["lr"] = K.get_value(self.model.optimizer.learning_rate)




In [17]:
num_epochs = 20
model = build_model()

optimizer = tf.keras.optimizers.Nadam(learning_rate = 0.001)
model.compile(loss = "sparse_categorical_crossentropy", optimizer = optimizer, metrics = ["accuracy"])
lr_scheduler = PiecewiseConstDecay([5, 15], [0.01, 0.005, 0.001])
piecewise_model_try = model.fit(X_train, y_train, epochs = num_epochs, validation_data = (X_valid, y_valid), callbacks = [lr_scheduler])

Epoch 1/20
1500/1500 [==============================] - 10s 4ms/step - loss: 3.4810 - accuracy: 0.6324 - val_loss: 0.9686 - val_accuracy: 0.6495 - lr: 0.0050
Epoch 2/20
1500/1500 [==============================] - 6s 4ms/step - loss: 0.7747 - accuracy: 0.7326 - val_loss: 0.6636 - val_accuracy: 0.7599 - lr: 0.0050
Epoch 3/20
1500/1500 [==============================] - 6s 4ms/step - loss: 0.8082 - accuracy: 0.6962 - val_loss: 0.7587 - val_accuracy: 0.7067 - lr: 0.0050
Epoch 4/20
1500/1500 [==============================] - 6s 4ms/step - loss: 1.0155 - accuracy: 0.5992 - val_loss: 0.9770 - val_accuracy: 0.5875 - lr: 0.0050
Epoch 5/20
1500/1500 [==============================] - 6s 4ms/step - loss: 1.1660 - accuracy: 0.5266 - val_loss: 1.1218 - val_accuracy: 0.5314 - lr: 0.0050
Epoch 6/20
1500/1500 [==============================] - 6s 4ms/step - loss: 1.0954 - accuracy: 0.5365 - val_loss: 1.0326 - val_accuracy: 0.5468 - lr: 0.0050
Epoch 7/20
1500/1500 [==============================] - 6

##performance Scheduling

In [19]:
model = build_model()
optimizer= tf.keras.optimizers.Nadam(learning_rate = 1e-3)
model.compile(loss = "sparse_categorical_crossentropy", optimizer = optimizer, metrics = ["accuracy"])

lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(factor = 0.5, patience = 5)
history = model.fit(X_train, y_train, epochs = num_epochs, validation_data = (X_valid, y_valid), callbacks = [lr_scheduler])

Epoch 1/20
1500/1500 [==============================] - 8s 4ms/step - loss: 3.3824 - accuracy: 0.7362 - val_loss: 1.1415 - val_accuracy: 0.7350 - lr: 0.0010
Epoch 2/20
1500/1500 [==============================] - 6s 4ms/step - loss: 0.7540 - accuracy: 0.7893 - val_loss: 0.6367 - val_accuracy: 0.8018 - lr: 0.0010
Epoch 3/20
1500/1500 [==============================] - 7s 5ms/step - loss: 0.5998 - accuracy: 0.8116 - val_loss: 0.6074 - val_accuracy: 0.8048 - lr: 0.0010
Epoch 4/20
1500/1500 [==============================] - 6s 4ms/step - loss: 0.5415 - accuracy: 0.8212 - val_loss: 0.5169 - val_accuracy: 0.8358 - lr: 0.0010
Epoch 5/20
1500/1500 [==============================] - 7s 5ms/step - loss: 0.4887 - accuracy: 0.8329 - val_loss: 0.5176 - val_accuracy: 0.8333 - lr: 0.0010
Epoch 6/20
1500/1500 [==============================] - 7s 5ms/step - loss: 0.4577 - accuracy: 0.8396 - val_loss: 0.4436 - val_accuracy: 0.8412 - lr: 0.0010
Epoch 7/20
1500/1500 [==============================] - 6s

#8. Practice training a deep neural network on the CIFAR10 image dataset:

>a. Build a DNN with 20 hidden layers of 100 neurons each (that’s
too many, but it’s the point of this exercise). Use He initialization
and the Swish activation function.

> b. Using Nadam optimization and early stopping, train the network
on the CIFAR10 dataset. You can load it with
tf.keras.datasets.cifar10.load_data(). The dataset is
composed of 60,000 32 × 32–pixel color images (50,000 for
training, 10,000 for testing) with 10 classes, so you’ll need a
softmax output layer with 10 neurons. Remember to search for
the right learning rate each time you change the model’s
architecture or hyperparameters.

> c. Now try adding batch normalization and compare the learning
curves: is it converging faster than before? Does it produce a
better model? How does it affect training speed?

> d. Try replacing batch normalization with SELU, and make the
necessary adjustments to ensure the network self-normalizes (i.e.,
standardize the input features, use LeCun normal initialization,
make sure the DNN contains only a sequence of dense layers,
etc.).

> e. Try regularizing the model with alpha dropout. Then, without
retraining your model, see if you can achieve better accuracy
using MC dropout.

> f. Retrain your model using 1cycle scheduling and see if it improves
training speed and model accuracy.

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
# %%capture
!pip install wandb

import wandb
from wandb.keras import WandbCallback
!wandb login

In [ ]:
data = tf.keras.datasets.cifar10.load_data()
(X_train_full, y_train_full), (X_test, y_test) = data
X_train_full.shape

In [ ]:
X_valid, X_train = X_train_full[:5000]/255, X_train_full[5000:]/255
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]
plt.imshow(X_train[0])

In [ ]:
tf.random.set_seed(24)
model = tf.keras.Sequential()
model.add(tf.keras.layers.Flatten(input_shape = [32,32,3]))
for i in range(20):
    model.add(tf.keras.layers.Dense(100, activation = 'swish', kernel_initializer = 'he_normal'))
model.add(tf.keras.layers.Dense(10, activation = 'softmax'))

In [ ]:
class CIFAR10(tf.keras.Model):
    def __init__(self, num_layers=20, num_units=100, activation="swish", kernel_initializer="he_normal", **kwargs):
        super().__init__()
        self.flatten = tf.keras.layers.Flatten()
        self.dense_layers = [tf.keras.layers.Dense(num_units, activation=activation, kernel_initializer=kernel_initializer)
                             for _ in range(num_layers)]
        self.output_layer = tf.keras.layers.Dense(10, activation="softmax")

    def call(self, input):
        x = self.flatten(input)
        for dense_layer in self.dense_layers:
            x = dense_layer(x)
        return self.output_layer(x)

cifar = CIFAR10()

In [ ]:
optimizer = tf.keras.optimizers.Nadam(learning_rate = 5e-5, beta_1 = 0.9, beta_2 = 0.999)

In [ ]:
wandb.init(
    # set the wandb project where this run will be logged
    project="chapter11-take-2",
    name = "question-a-5e-5lr-sub",

    # track hyperparameters and run metadata
    config={
    "learning_rate": 5e-5,
    "architecture": "Basic DNN",
    "dataset": "CIFAR-10",
    "epochs": 20,
    "loss" : "sparse_categorical_crossentropy",

    }
)

config = wandb.config

In [ ]:
cifar.compile(loss = config.loss, optimizer = optimizer, metrics = ["accuracy"])

In [ ]:
wandb.init(
    # set the wandb project where this run will be logged
    project="chapter11-take-2",
    name = "question-a-5e-5lr-bookish",

    # track hyperparameters and run metadata
    config={
    "learning_rate": 5e-5,
    "architecture": "Basic DNN",
    "dataset": "CIFAR-10",
    "epochs": 20,
    "loss": "sparse_categorical_crossentropy"
    }
)

config = wandb.config

In [ ]:
model.compile(loss = config.loss, optimizer = optimizer, metrics = ["accuracy"])

In [ ]:
early_stop = tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 5)

In [ ]:
history = cifar.fit(X_train,y_train, epochs = 20, validation_data = (X_valid, y_valid), callbacks = [early_stop, WandbCallback()])

## With normalization


In [ ]:
class CIFAR10Normalized(tf.keras.Model):
    def __init__(self, activation = "swish", num_units=100, kernel_initializer = "he_normal", num_layer = 20, **kwargs):
        super().__init__(**kwargs)
        self.flatten = tf.keras.layers.Flatten(input_shape = [32,32]) # input_size, 1024
        self.norm = [tf.keras.layers.BatchNormalization() for i in range(num_layer)]
        self.dense = [tf.keras.layers.Dense(num_units, kernel_initializer = kernel_initializer) for i in range(num_layer)]# 100, 1024 and 100,100
        self.activation = [tf.keras.layers.Activation(activation) for i in range(num_layer)]
        self.op = tf.keras.layers.Dense(10, activation = "softmax") #100, 10

    def call(self, input):
        input = self.flatten(input)
        for i, j, k in zip(self.dense, self.norm, self.activation):
            input = j(input)
            input = i(input)
            input = k(input)
            # print(self.op(input))
        return self.op(input)

lass0 = CIFAR10Normalized()
lass0


In [ ]:
optimizer = tf.keras.optimizers.Nadam(learning_rate = 5e-4, beta_1 = 0.9, beta_2= 0.999)

In [ ]:
wandb.init(
    # set the wandb project where this run will be logged
    project="Chapter_11_CIFAR10_c",
    name = "question-c-5e-4lr",

    # track hyperparameters and run metadata
    config={
    "learning_rate": 5e-4,
    "architecture": "Basic DNN",
    "dataset": "CIFAR-10",
    "epochs": 20,
    }
)

config = wandb.config

In [ ]:
lass0.compile(loss = "sparse_categorical_crossentropy", optimizer = optimizer, metrics = ["accuracy"])

In [ ]:
history = lass0.fit(X_train, y_train, epochs = config.epochs, validation_data = (X_test, y_test), callbacks = [early_stop, WandbCallback()])

Epoch 1/20
1406/1407 [============================>.] - ETA: 0s - loss: 2.2365 - accuracy: 0.1792

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
wandb: ERROR Can't save model in the h5py format. The model will be saved as as an W&B Artifact in the 'tf' format.
wandb: Adding directory to artifact (/content/wandb/run-20240327_105412-em6po592/files/model-best)... Done. 0.0s


1407/1407 [==============================] - 73s 38ms/step - loss: 2.2364 - accuracy: 0.1792 - val_loss: 548.0206 - val_accuracy: 0.1180
Epoch 2/20
1407/1407 [==============================] - ETA: 0s - loss: 1.9394 - accuracy: 0.2929

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
wandb: ERROR Can't save model in the h5py format. The model will be saved as as an W&B Artifact in the 'tf' format.
wandb: Adding directory to artifact (/content/wandb/run-20240327_105412-em6po592/files/model-best)... Done. 0.0s


1407/1407 [==============================] - 54s 38ms/step - loss: 1.9394 - accuracy: 0.2929 - val_loss: 234.6631 - val_accuracy: 0.1474
Epoch 3/20
1407/1407 [==============================] - ETA: 0s - loss: 1.8218 - accuracy: 0.3432

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
wandb: ERROR Can't save model in the h5py format. The model will be saved as as an W&B Artifact in the 'tf' format.
wandb: Adding directory to artifact (/content/wandb/run-20240327_105412-em6po592/files/model-best)... Done. 0.0s


1407/1407 [==============================] - 53s 38ms/step - loss: 1.8218 - accuracy: 0.3432 - val_loss: 120.0418 - val_accuracy: 0.1567
Epoch 4/20
1406/1407 [============================>.] - ETA: 0s - loss: 1.7416 - accuracy: 0.3791

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
wandb: ERROR Can't save model in the h5py format. The model will be saved as as an W&B Artifact in the 'tf' format.
wandb: Adding directory to artifact (/content/wandb/run-20240327_105412-em6po592/files/model-best)... Done. 0.0s


1407/1407 [==============================] - 54s 38ms/step - loss: 1.7417 - accuracy: 0.3790 - val_loss: 65.6545 - val_accuracy: 0.1509
Epoch 5/20
1407/1407 [==============================] - 44s 31ms/step - loss: 1.6920 - accuracy: 0.3984 - val_loss: 398.8511 - val_accuracy: 0.1041
Epoch 6/20
1407/1407 [==============================] - 48s 34ms/step - loss: 1.6380 - accuracy: 0.4163 - val_loss: 168.4453 - val_accuracy: 0.1374
Epoch 7/20
1407/1407 [==============================] - 44s 31ms/step - loss: 1.5981 - accuracy: 0.4338 - val_loss: 181.4198 - val_accuracy: 0.1542
Epoch 8/20
1407/1407 [==============================] - 43s 31ms/step - loss: 1.5652 - accuracy: 0.4459 - val_loss: 267.4036 - val_accuracy: 0.1639
Epoch 9/20
1407/1407 [==============================] - 52s 37ms/step - loss: 1.5299 - accuracy: 0.4589 - val_loss: 186.8361 - val_accuracy: 0.1566


##SELU, leCun norm for self normalization

In [ ]:
class WithSeLU(tf.keras.Model):
    def __init__(self, activation = "selu",num_layers = 20, num_units = 100, kernel_initializer = "lecun_normal", **kwargs):
        super().__init__(**kwargs)
        self.flatten = tf.keras.layers.Flatten()
        self.dense = [tf.keras.layers.Dense(num_units, activation = activation, kernel_initializer = kernel_initializer) for i in range(num_layers)]
        # self.norm - [tf.keras.layers.Normalization()]
        self.op = tf.keras.layers.Dense(10, activation = "softmax")

    def call(self, input):
        input = self.flatten(input)
        for i in self.dense:
            input = i(input)
        return self.op(input)
with_selu = WithSeLU()

In [ ]:
import os
import random
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
os.environ['No_IDEA'] = '1'
random.seed(hash("settng random seeds")%2**32-1)
np.random.seed(hash("Improves reproducability")%2**32-1)
tf.random.set_seed(hash("removes stochasticity")%2**32-1)

In [ ]:
wandb.init(
    # set the wandb project where this run will be logged
    project="Chapter_11_CIFAR10_d",
    name = "question-d-7e-4lr-nadam",

    # track hyperparameters and run metadata
    config={
    "learning_rate": 7e-4,
    "architecture": "Basic DNN",
    "dataset": "CIFAR-10",
    "epochs": 90,
    }
)

config = wandb.config

accuracy,▁▃▃▄▄▄▅▅▅▅▅▆▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇████
epoch,▁▁▁▂▂▂▂▂▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▇▇▇▇▇███
loss,█▇▆▅▅▅▄▄▄▄▄▄▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁
val_accuracy,▁▂▄▄▅▅▄▆▆▅▆▇▆▇▇▇▆▇▇▇▇▇▇████▇█▇████
val_loss,█▆▅▅▄▄▄▃▃▄▃▂▄▂▁▂▃▁▂▁▁▂▁▁▂▂▂▂▁▃▂▂▂▂
accuracy,0.62053
best_epoch,23
best_val_loss,1.42329
epoch,33
loss,1.06064
val_accuracy,0.501


In [ ]:
early = tf.keras.callbacks.EarlyStopping(patience = 10, monitor = "val_loss")
# optimizer = tf.keras.optimizers.SGD(learning_rate = config.learning_rate, momentum = 0.9, nesterov = True)
optimizer = tf.keras.optimizers.Nadam(learning_rate = config.learning_rate)
with_selu.compile(loss = "sparse_categorical_crossentropy", optimizer = optimizer, metrics = ["accuracy"])
history = with_selu.fit(X_train,y_train, epochs = config.epochs, validation_data = (X_valid, y_valid), callbacks = [WandbCallback(), early])

Epoch 1/90
1407/1407 [==============================] - ETA: 0s - loss: 2.0176 - accuracy: 0.2534

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
wandb: ERROR Can't save model in the h5py format. The model will be saved as as an W&B Artifact in the 'tf' format.
wandb: Adding directory to artifact (/content/wandb/run-20240327_112954-a2efv0wi/files/model-best)... Done. 0.0s


1407/1407 [==============================] - 28s 14ms/step - loss: 2.0176 - accuracy: 0.2534 - val_loss: 1.9142 - val_accuracy: 0.2942
Epoch 2/90
1405/1407 [============================>.] - ETA: 0s - loss: 1.8511 - accuracy: 0.3269

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
wandb: ERROR Can't save model in the h5py format. The model will be saved as as an W&B Artifact in the 'tf' format.
wandb: Adding directory to artifact (/content/wandb/run-20240327_112954-a2efv0wi/files/model-best)... Done. 0.0s


1407/1407 [==============================] - 20s 14ms/step - loss: 1.8508 - accuracy: 0.3270 - val_loss: 1.8541 - val_accuracy: 0.3378
Epoch 3/90
1403/1407 [============================>.] - ETA: 0s - loss: 1.7926 - accuracy: 0.3534

...

1407/1407 [==============================] - 20s 14ms/step - loss: 1.4540 - accuracy: 0.4827 - val_loss: 1.5425 - val_accuracy: 0.4540
Epoch 23/90
1403/1407 [============================>.] - ETA: 0s - loss: 1.4408 - accuracy: 0.4871

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
wandb: ERROR Can't save model in the h5py format. The model will be saved as as an W&B Artifact in the 'tf' format.
wandb: Adding directory to artifact (/content/wandb/run-20240327_112954-a2efv0wi/files/model-best)... Done. 0.0s


1407/1407 [==============================] - 20s 14ms/step - loss: 1.4405 - accuracy: 0.4872 - val_loss: 1.4947 - val_accuracy: 0.4632
Epoch 24/90
1407/1407 [==============================] - 17s 12ms/step - loss: 1.4207 - accuracy: 0.4964 - val_loss: 1.5205 - val_accuracy: 0.4640
Epoch 25/90
1407/1407 [==============================] - 16s 12ms/step - loss: 1.3997 - accuracy: 0.5038 - val_loss: 1.5515 - val_accuracy: 0.4504
Epoch 26/90
1405/1407 [============================>.] - ETA: 0s - loss: 1.4316 - accuracy: 0.4922

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
wandb: ERROR Can't save model in the h5py format. The model will be saved as as an W&B Artifact in the 'tf' format.
wandb: Adding directory to artifact (/content/wandb/run-20240327_112954-a2efv0wi/files/model-best)... Done. 0.0s


1407/1407 [==============================] - 23s 16ms/step - loss: 1.4316 - accuracy: 0.4922 - val_loss: 1.4756 - val_accuracy: 0.4856
Epoch 27/90
1407/1407 [==============================] - 16s 12ms/step - loss: 1.3783 - accuracy: 0.5112 - val_loss: 1.5023 - val_accuracy: 0.4676
Epoch 28/90
1407/1407 [==============================] - 16s 11ms/step - loss: 1.3585 - accuracy: 0.5188 - val_loss: 1.5026 - val_accuracy: 0.4688
Epoch 29/90
1407/1407 [==============================] - 16s 11ms/step - loss: 1.3480 - accuracy: 0.5208 - val_loss: 1.5183 - val_accuracy: 0.4720
Epoch 30/90
1407/1407 [==============================] - 16s 12ms/step - loss: 1.3348 - accuracy: 0.5272 - val_loss: 1.5487 - val_accuracy: 0.4528
Epoch 31/90
1407/1407 [==============================] - 17s 12ms/step - loss: 1.3208 - accuracy: 0.5303 - val_loss: 1.5229 - val_accuracy: 0.4688
Epoch 32/90
1407/1407 [==============================] - 16s 11ms/step - loss: 2.4072 - accuracy: 0.4093 - val_loss: 1.7952 - val_

## Alpha Dropout and then MC dropout without retrain

###AlphaDropout

In [ ]:
wandb.init(
    project = "Chapter_11_CIFAR_10_e",
    name = "question_e_alpha_dropout_lr5e-4",

    config = {
        "dataset": "CIFAR_10",
        "lr": 5e-4,
        "archetecture": "Basic DNN",
        "loss": "sparse_categorical_crossentropy",
        "epochs": 20,
        "num_layer":20,
        "num_units": 100,
        "activation": "selu",
        "kernel_init": "lecun_normal"
    }
)

wandb: Currently logged in as: 1999sandeepdey (thecommon). Use `wandb login --relogin` to force relogin


In [ ]:
config  = wandb.config

In [ ]:
class AlphaSeLU(tf.keras.Model):
    def __init__(self, num_layers = config.num_layer, num_units = config.num_units, activation = config.activation, kernel_initializer = config.kernel_init, **kwargs):
        super().__init__(**kwargs)
        self.flatten = tf.keras.layers.Flatten(input_shape = [32, 32, 3])
        self.dense = [tf.keras.layers.Dense(num_units, kernel_initializer = kernel_initializer, activation = activation) for i in range(num_layers)]
        self.dropout = tf.keras.layers.AlphaDropout(rate = 0.1)
        self.op = tf.keras.layers.Dense(10, activation = "softmax")

    def call(self, input):
        input = self.flatten(input)
        for dense in self.dense:
            input = dense(input)

        input = self.dropout(input)
        return self.op(input)

model = AlphaSeLU(name = "Alpha_model")

optimizer = tf.keras.optimizers.Nadam(learning_rate = config.lr, beta_1 = 0.9, beta_2 = 0.999)

early_stop = tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights = True, monitor = "val_loss")
model_check = tf.keras.callbacks.ModelCheckpoint("aplha_model", save_best_only = True)

model.compile(loss = config.loss, optimizer = optimizer, metrics = ['accuracy'])

history = model.fit(X_train, y_train, epochs = config.epochs, validation_data = (X_valid, y_valid), callbacks = [early_stop, model_check, WandbCallback()])


Epoch 1/20
1407/1407 [==============================] - 33s 17ms/step - loss: 2.0061 - accuracy: 0.2685 - val_loss: 1.8834 - val_accuracy: 0.3216
Epoch 2/20
1407/1407 [==============================] - 23s 16ms/step - loss: 1.8437 - accuracy: 0.3345 - val_loss: 1.9484 - val_accuracy: 0.3144
Epoch 3/20
1407/1407 [==============================] - 22s 16ms/step - loss: 1.7809 - accuracy: 0.3555 - val_loss: 1.9312 - val_accuracy: 0.3292
Epoch 4/20
1407/1407 [==============================] - 27s 19ms/step - loss: 1.7293 - accuracy: 0.3779 - val_loss: 1.7517 - val_accuracy: 0.3812
Epoch 5/20
1407/1407 [==============================] - 24s 17ms/step - loss: 1.6814 - accuracy: 0.3971 - val_loss: 1.6913 - val_accuracy: 0.4140
Epoch 6/20
1407/1407 [==============================] - 21s 15ms/step - loss: 1.6450 - accuracy: 0.4136 - val_loss: 1.6701 - val_accuracy: 0.4056
Epoch 7/20
1407/1407 [==============================] - 19s 14ms/step - loss: 1.6143 - accuracy: 0.4264 - val_loss: 1.6123 -

In [ ]:
model.evaluate(X_test, y_test)

313/313 [==============================] - 2s 5ms/step - loss: 101.0467 - accuracy: 0.2959


[101.04667663574219, 0.29589998722076416]

In [ ]:
class AlphaSeLU(tf.keras.Model):
    def __init__(self, num_layers = 20, num_units = 100, activation = 'selu', kernel_initializer = "lecun_normal", rate = 0.1, **kwargs):
        super().__init__(**kwargs)
        self.flatten = tf.keras.layers.Flatten()
        self.dense = [tf.keras.layers.Dense(num_units, activation = activation,kernel_initializer = kernel_initializer) for i in range(num_layers)]
        self.dropout = tf.keras.layers.AlphaDropout(rate = rate)
        self.op = tf.keras.layers.Dense(10, activation = "softmax")

    def call(self, input):
        input = self.flatten(input)
        for dense in self.dense:
            # input = dropout(input)
            input = dense(input)
        input = self.dropout(input)
        return self.op(input)

alpha = AlphaSeLU()

In [ ]:
wandb.init(
    # set the wandb project where this run will be logged
    project="Chapter_11_CIFAR10_e",
    name = "e_1e-3lr_nestrov"

    # track hyperparameters and run metadata
    config={
    "learning_rate": 1e-3,
    "architecture": "Basic DNN",
    "dataset": "CIFAR-10",
    "epochs": 90,
    "loss": "sparse_categorical_crossentropy"
    }
)

config = wandb.config

In [ ]:
optimizer = tf.keras.optimizers.SGD(learning_rate = config.learning_rate, momentum = 0.9, nesterov = True)

alpha.compile(loss = config.loss, optimizer = optimizer, metrics = ["accuracy"])

history = alpha.fit(X_train, y_train, epochs = config.epochs, validation_data = (X_valid, y_valid), callbacks = [WandbCallback()])

Epoch 1/90
1402/1407 [============================>.] - ETA: 0s - loss: 1.9384 - accuracy: 0.2913

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
wandb: ERROR Can't save model in the h5py format. The model will be saved as as an W&B Artifact in the 'tf' format.
wandb: Adding directory to artifact (/content/wandb/run-20240327_121434-i3zylg43/files/model-best)... Done. 0.0s


1407/1407 [==============================] - 17s 10ms/step - loss: 1.9382 - accuracy: 0.2913 - val_loss: 1.7850 - val_accuracy: 0.3548
Epoch 2/90
1407/1407 [==============================] - 15s 11ms/step - loss: 1.7591 - accuracy: 0.3608 - val_loss: 1.9032 - val_accuracy: 0.3406
Epoch 3/90
1403/1407 [============================>.] - ETA: 0s - loss: 1.6825 - accuracy: 0.3898

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
wandb: ERROR Can't save model in the h5py format. The model will be saved as as an W&B Artifact in the 'tf' format.
wandb: Adding directory to artifact (/content/wandb/run-20240327_121434-i3zylg43/files/model-best)... Done. 0.1s


1407/1407 [==============================] - 19s 14ms/step - loss: 1.6821 - accuracy: 0.3898 - val_loss: 1.6845 - val_accuracy: 0.4014
Epoch 4/90
1403/1407 [============================>.] - ETA: 0s - loss: 1.6255 - accuracy: 0.4138

...

1407/1407 [==============================] - 14s 10ms/step - loss: 1.2807 - accuracy: 0.5382 - val_loss: 1.4486 - val_accuracy: 0.4918
Epoch 20/90
1405/1407 [============================>.] - ETA: 0s - loss: 1.2639 - accuracy: 0.5430

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
wandb: ERROR Can't save model in the h5py format. The model will be saved as as an W&B Artifact in the 'tf' format.
wandb: Adding directory to artifact (/content/wandb/run-20240327_121434-i3zylg43/files/model-best)... Done. 0.0s


1407/1407 [==============================] - 15s 10ms/step - loss: 1.2642 - accuracy: 0.5429 - val_loss: 1.4411 - val_accuracy: 0.4918
Epoch 21/90
1407/1407 [==============================] - 12s 8ms/step - loss: 1.2495 - accuracy: 0.5486 - val_loss: 1.4539 - val_accuracy: 0.5050
Epoch 22/90
1407/1407 [==============================] - 11s 8ms/step - loss: 1.2337 - accuracy: 0.5566 - val_loss: 1.4682 - val_accuracy: 0.4954
Epoch 23/90
1407/1407 [==============================] - 12s 8ms/step - loss: 1.2201 - accuracy: 0.5603 - val_loss: 1.4854 - val_accuracy: 0.4962
Epoch 24/90
1400/1407 [============================>.] - ETA: 0s - loss: 1.2052 - accuracy: 0.5660

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
wandb: ERROR Can't save model in the h5py format. The model will be saved as as an W&B Artifact in the 'tf' format.
wandb: Adding directory to artifact (/content/wandb/run-20240327_121434-i3zylg43/files/model-best)... Done. 0.0s


1407/1407 [==============================] - 16s 11ms/step - loss: 1.2048 - accuracy: 0.5662 - val_loss: 1.4410 - val_accuracy: 0.5170
Epoch 25/90
1407/1407 [==============================] - 12s 8ms/step - loss: 1.1861 - accuracy: 0.5722 - val_loss: 1.4553 - val_accuracy: 0.5194
Epoch 26/90
1407/1407 [==============================] - 11s 8ms/step - loss: 1.1732 - accuracy: 0.5779 - val_loss: 1.5093 - val_accuracy: 0.5010
Epoch 27/90
1407/1407 [==============================] - 11s 8ms/step - loss: 1.1635 - accuracy: 0.5804 - val_loss: 1.5136 - val_accuracy: 0.5106
Epoch 28/90
1407/1407 [==============================] - 11s 8ms/step - loss: 1.1512 - accuracy: 0.5867 - val_loss: 1.5328 - val_accuracy: 0.5058
Epoch 29/90
1407/1407 [==============================] - 11s 8ms/step - loss: 1.1363 - accuracy: 0.5892 - val_loss: 1.5197 - val_accuracy: 0.5174
Epoch 30/90
1407/1407 [==============================] - 12s 8ms/step - loss: 1.1229 - accuracy: 0.5953 - val_loss: 1.6237 - val_accura

###MCDropout

In [ ]:
wandb.init(
    project = "Chapter_11_question_e_MC",
    name = "Question_e_mc_like_prev",

    config = {
        "epochs": 20,
        "archetechture": "Simple_DDN_MC",
        "data_set": "cifar_10",
        "loss": "sparse_categorical_crossentropy"
    }
)

In [ ]:
class MCDrop(tf.keras.layers.AlphaDropout):
    def __init__(self, rate, **kwargs):
        super().__init__(rate, **kwargs)

    def call(self, inputs, training = False):
        return super().call(inputs, training = True)

early = tf.keras.callbacks.EarlyStopping(monitor = "val_loss", patience = 5)


mc_model = tf.keras.Sequential([
    (
        MCDrop(0.1)
        if isinstance(layer, tf.keras.layers.AlphaDropout)
        else layer
    )
    for layer in model.layers
])

def mc_dropout_predict_probas(model, X, n_samples = 10):
    Y_probas = [model.predict(X) for sample in range(n_samples)]
    return np.mean(Y_probas, axis = 0)

def mc_dropout_predict_classes(model, X, n_samples = 10):
    Y_probas = mc_dropout_predict_probas(model, X, n_samples)
    return Y_probas.argmax(axis = 1)

mc_model.compile(loss = config.loss, optimizer = optimizer, metrics = ["accuracy"])

tf.random.set_seed(37)

history = mc_model.fit(X_train, y_train, epochs = config.epochs, validation_data = (X_valid, y_valid), callbacks = [WandbCallback(), early_stop])

y_pred = mc_dropout_predict_classes(mc_model, X_valid)
accuracy = (y_pred == y_valid[:, 0]).mean()



Epoch 1/20
1404/1407 [============================>.] - ETA: 0s - loss: 1.9147 - accuracy: 0.2952

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
wandb: Adding directory to artifact (/content/wandb/run-20240409_120429-cchc08wb/files/model-best)... Done. 0.0s


1407/1407 [==============================] - 23s 14ms/step - loss: 1.9144 - accuracy: 0.2953 - val_loss: 1.8563 - val_accuracy: 0.3222
Epoch 2/20
1406/1407 [============================>.] - ETA: 0s - loss: 1.8280 - accuracy: 0.3400

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
wandb: Adding directory to artifact (/content/wandb/run-20240409_120429-cchc08wb/files/model-best)... Done. 0.0s


1407/1407 [==============================] - 20s 14ms/step - loss: 1.8279 - accuracy: 0.3400 - val_loss: 1.8414 - val_accuracy: 0.3202
Epoch 3/20
1405/1407 [============================>.] - ETA: 0s - loss: 1.7724 - accuracy: 0.3622

...

1407/1407 [==============================] - 20s 14ms/step - loss: 1.4458 - accuracy: 0.4900 - val_loss: 1.5423 - val_accuracy: 0.4644
Epoch 15/20
1406/1407 [============================>.] - ETA: 0s - loss: 1.4293 - accuracy: 0.4937

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
wandb: Adding directory to artifact (/content/wandb/run-20240409_120429-cchc08wb/files/model-best)... Done. 0.0s


1407/1407 [==============================] - 22s 16ms/step - loss: 1.4293 - accuracy: 0.4937 - val_loss: 1.4971 - val_accuracy: 0.4614
Epoch 16/20
1407/1407 [==============================] - 17s 12ms/step - loss: 1.4102 - accuracy: 0.5020 - val_loss: 1.5279 - val_accuracy: 0.4706
Epoch 17/20
1407/1407 [==============================] - ETA: 0s - loss: 1.3983 - accuracy: 0.5057

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
wandb: Adding directory to artifact (/content/wandb/run-20240409_120429-cchc08wb/files/model-best)... Done. 0.0s


1407/1407 [==============================] - 23s 17ms/step - loss: 1.3983 - accuracy: 0.5057 - val_loss: 1.4776 - val_accuracy: 0.4828
Epoch 18/20
1407/1407 [==============================] - 17s 12ms/step - loss: 1.3782 - accuracy: 0.5100 - val_loss: 1.5110 - val_accuracy: 0.4676
Epoch 19/20
1407/1407 [==============================] - 23s 16ms/step - loss: 1.3660 - accuracy: 0.5151 - val_loss: 1.5225 - val_accuracy: 0.4810
Epoch 20/20
157/157 [==============================] - 0s 3ms/step


In [ ]:
accuracy

0.4564

##1cycle rescheduling

In [ ]:
wandb.init(
    project = "Chapter_11_q_f_onecycle",
    name = "question_f_learing_rate_4e-5",

    config = {
        "archetecture": "MLP_simple_DNN",
        "dataset" : "cifar10",
        "loss":"sparse_categorical_crossentropy",
        "epochs": 20,
        "learning_rate": 1e-3
    }
)

In [ ]:
config = wandb.config

In [ ]:
class OneCycle(tf.keras.Model):
    def __init__(self, rate = 0.1, num_layers = 20, num_units = 100, activation = "selu", kernel_initializer = "lecun_normal", **kwargs):
        super().__init__(**kwargs)
        self.flatten = tf.keras.layers.Flatten()
        self.dense = [tf.keras.layers.Dense(num_units, activation = activation, kernel_initializer = kernel_initializer) for i in range(num_layers)]
        self.dropout = tf.keras.layers.AlphaDropout(rate = rate)
        self.op = tf.keras.layers.Dense(10, activation = "softmax")

    def call(self, input):
        input = self.flatten(input)
        for dense in self.dense:
            input = dense(input)
        input = self.dropout(input)
        return self.op(input)

model = OneCycle()

In [ ]:
optimizer = tf.keras.optimizers.Nadam(learning_rate = config.learning_rate, beta_1 = 0.9, beta_2 = 0.999)

In [ ]:
early_stop = tf.keras.callbacks.EarlyStopping(patience = 5, monitor = "val_loss")
model_checkp = tf.keras.callbacks.ModelCheckpoint("onecycle", save_best_only = True)

In [ ]:
model.compile(loss = config.loss, optimizer = optimizer, metrics = ["accuracy"])

In [ ]:
class OneCycleScheduler(tf.keras.callbacks.Callback):
    def __init__(self, iterations, max_lr=1e-3, start_lr=None,
                 last_iterations=None, last_lr=None):
        self.iterations = iterations
        self.max_lr = max_lr
        self.start_lr = start_lr or max_lr / 10
        self.last_iterations = last_iterations or iterations // 10 + 1
        self.half_iteration = (iterations - self.last_iterations) // 2
        self.last_lr = last_lr or self.start_lr / 1000
        self.iteration = 0

    def _interpolate(self, iter1, iter2, lr1, lr2):
        return (lr2 - lr1) * (self.iteration - iter1) / (iter2 - iter1) + lr1

    def on_batch_begin(self, batch, logs):
        if self.iteration < self.half_iteration:
            lr = self._interpolate(0, self.half_iteration, self.start_lr,
                                   self.max_lr)
        elif self.iteration < 2 * self.half_iteration:
            lr = self._interpolate(self.half_iteration, 2 * self.half_iteration,
                                   self.max_lr, self.start_lr)
        else:
            lr = self._interpolate(2 * self.half_iteration, self.iterations,
                                   self.start_lr, self.last_lr)
        self.iteration += 1
        K.set_value(self.model.optimizer.learning_rate, lr)

In [ ]:
import math
batch_size = 28
onecycle = OneCycleScheduler(math.ceil(len(X_train)/batch_size) * config.epochs, max_lr = 1e-3)

In [ ]:
history = model.fit(X_train, y_train, epochs = config.epochs, validation_data = (X_valid, y_valid), callbacks = [WandbCallback(), early_stop, model_checkp, ])

Epoch 1/20
1407/1407 [==============================] - ETA: 0s - loss: 2.1345 - accuracy: 0.1785

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
wandb: ERROR Can't save model in the h5py format. The model will be saved as as an W&B Artifact in the 'tf' format.
wandb: Adding directory to artifact (/content/wandb/run-20240409_145540-h88mqzdg/files/model-best)... Done. 0.0s


1407/1407 [==============================] - 36s 17ms/step - loss: 2.1345 - accuracy: 0.1785 - val_loss: 2.1651 - val_accuracy: 0.2088
Epoch 2/20
1405/1407 [============================>.] - ETA: 0s - loss: 1.9877 - accuracy: 0.2333

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
wandb: ERROR Can't save model in the h5py format. The model will be saved as as an W&B Artifact in the 'tf' format.
wandb: Adding directory to artifact (/content/wandb/run-20240409_145540-h88mqzdg/files/model-best)... Done. 0.0s


1407/1407 [==============================] - 23s 17ms/step - loss: 1.9875 - accuracy: 0.2334 - val_loss: 1.9471 - val_accuracy: 0.2498
Epoch 3/20
1407/1407 [==============================] - 16s 11ms/step - loss: 1.9543 - accuracy: 0.2518 - val_loss: 1.9586 - val_accuracy: 0.2548
Epoch 4/20
1406/1407 [============================>.] - ETA: 0s - loss: 1.9176 - accuracy: 0.2653

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
wandb: ERROR Can't save model in the h5py format. The model will be saved as as an W&B Artifact in the 'tf' format.
wandb: Adding directory to artifact (/content/wandb/run-20240409_145540-h88mqzdg/files/model-best)... Done. 0.0s


1407/1407 [==============================] - 22s 16ms/step - loss: 1.9177 - accuracy: 0.2653 - val_loss: 1.8990 - val_accuracy: 0.2854
Epoch 5/20
1403/1407 [============================>.] - ETA: 0s - loss: 1.9180 - accuracy: 0.2717

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
wandb: ERROR Can't save model in the h5py format. The model will be saved as as an W&B Artifact in the 'tf' format.
wandb: Adding directory to artifact (/content/wandb/run-20240409_145540-h88mqzdg/files/model-best)... Done. 0.0s


1407/1407 [==============================] - 22s 16ms/step - loss: 1.9184 - accuracy: 0.2716 - val_loss: 1.8757 - val_accuracy: 0.2868
Epoch 6/20
1407/1407 [==============================] - 16s 12ms/step - loss: 1.9477 - accuracy: 0.2557 - val_loss: 1.9652 - val_accuracy: 0.2546
Epoch 7/20
1407/1407 [==============================] - 16s 12ms/step - loss: 1.8997 - accuracy: 0.2692 - val_loss: 1.9161 - val_accuracy: 0.2750
Epoch 8/20
1407/1407 [==============================] - 16s 12ms/step - loss: 2.1227 - accuracy: 0.2185 - val_loss: 2.0413 - val_accuracy: 0.2070
Epoch 9/20
1407/1407 [==============================] - 16s 12ms/step - loss: 1.9840 - accuracy: 0.2121 - val_loss: 2.1271 - val_accuracy: 0.2122
Epoch 10/20
1407/1407 [==============================] - 17s 12ms/step - loss: 2.1837 - accuracy: 0.1796 - val_loss: 2.1198 - val_accuracy: 0.1682
